In [3]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
import numpy as np

# Drop indexs more from lower score and less from higher score
# https://stackoverflow.com/questions/28556942/pandas-remove-rows-at-random-without-shuffling-dataset/28557333#28557333
# np.random.seed(10)

feature_list = ["topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count"]

#display(data)

X = df[feature_list]
y = df[["label"]]

# X = df[df.label > 1000][feature_list]
# y = df[df.label > 1000][["label"]]
# print(y.shape)
# print(X.shape)

# Bin all of y first
# uniform = All bins in each feature have identical widths.
# quantile = All bins in each feature have the same number of points.
# kmeans = Values in each bin have the same nearest center of a 1D k-means cluster.
bins = 5
est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy="uniform") #
est.fit(y)
yt = pd.DataFrame(data=est.transform(y), columns=["y"])

display(est.transform(y))

y_filtered_drop = yt[yt.y == 0][:9900]
yt_filtered = yt.drop(index=y_filtered_drop.index)
X_filtered = X.drop(index=y_filtered_drop.index)

#print(y_filtered_drop.shape)
#print(yt_filtered.shape)
#print(X_filtered.shape)

display(yt_filtered["y"].value_counts())


#df["est_label"] = yt
#display(pd.DataFrame(data=yt, columns=["y"])["y"].value_counts())
#display(df[[*feature_list, "label", "est_label"]])
#display(df["label"])


# Bad idea to do random split with so skewed data   
#data = scaler.transform(data)
#X_train, X_validate, y_train, y_validate = train_test_split(X, yt, test_size=0.7) 


# https://stats.stackexchange.com/questions/131255/class-imbalance-in-supervised-machine-learning
# https://datascience.stackexchange.com/questions/32818/train-test-split-of-unbalanced-dataset-classification/32820#32820
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
skf = StratifiedKFold(n_splits=50, random_state=42, shuffle=True)
# skf.get_n_splits(X, yt)
balanced_accuracies = []
accuracies = []
predictions = np.array([])
true_values = np.array([])

for train_index, test_index in skf.split(X_filtered, yt_filtered):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_validate = X_filtered.iloc[train_index], X_filtered.iloc[test_index]
    y_train, y_true = yt_filtered.iloc[train_index], yt_filtered.iloc[test_index]

    #scaler = MinMaxScaler(min=0, max=1)
    scaler = MinMaxScaler()
    scaler.fit(X_train, y_train) # Don't cheat: fit only training data. y is ignored, but needs same shape[rows]

    model = MultinomialNB()

    # Train the model using the training sets - scale only features (?)
    model.fit(scaler.transform(X_train), y_train)

    # Predic the model
    y_pred = model.predict(scaler.transform(X_validate))

    # At this point, the model can classify all 0 positively, but no more
    accuracy = metrics.accuracy_score(y_true, y_pred, normalize=True)
    # TODO: balanced - https://stackoverflow.com/questions/55838262/why-does-cross-validation-give-consistently-higher-scores-than-normal-fitting-an
    balanced_accuracy = metrics.balanced_accuracy_score(y_true, y_pred, adjusted=False)
    # TODO: https://stats.stackexchange.com/questions/99667/naive-bayes-with-unbalanced-classes

    # display
    
    accuracies.append(accuracy)
    balanced_accuracies.append(balanced_accuracy)
    predictions = np.append(predictions, y_pred)
    true_values = np.append(true_values, y_true)
    # print(accuracy)

    # accuracy_zeros = metrics.accuracy_score(y_validate, [0]*len(y_validate), normalize=True)
    # print(accuracy_zeros)

# TODO: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html

print("\n\nMean:", np.mean(accuracies))
print("\n\nMean balanced:", np.mean(balanced_accuracies))
results = pd.DataFrame(data={"pred": predictions, "true": true_values})
counts = pd.DataFrame(data={"pred": results["pred"].value_counts(), "true": results["true"].value_counts()})
display(counts)
#results["pred_values"].value_counts()
#results["true_values"].value_counts()
#display(results[["pred_values", "true_values"]])


KeyError: "None of [Index(['topic_similarity', 'tfidf_score', 'words_count', 'stop_words_count',\n       'bad_words_count'],\n      dtype='object')] are in the [columns]"